In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Tutorial: Hyperparameter Search

In [3]:
import torch
import pytorch_lightning as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
#from classification.models.MelSpectrogramCNN_8K import MelSpectrogramCNN_8KPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule

from classification.models.SpectrogramCNN_8K_Dataset2 import SpectrogramCNN_8K_Dataset2_PLModule
from classification.models.CRNN_8k_D2 import CRNN8k_D2_PLModule
from classification.trainer.HyperParamSearch import MetricsCallback, save_model

from classification.models.AblationModel import AblationModelPLModule

In [4]:
from pytorch_lightning.callbacks import Callback

class SaveBestCallback(Callback): 
    def __init__(self, model_name = "newest_model", add_v_number = True):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None
        self.add_v_number = add_v_number

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            #save_path = self.model_name + (( "_v{}".format(trainer.logger.version) +  "_best.p") if self.add_v_number else "")
            save_path = self.model_name + str(pl_module.val_results_history[-1]["val_acc"]) + ".p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveBestCallback("optuna_ablationNormal_", add_v_number = False)

### Objective

In [5]:
def objective(trial):
    metrics_callback = MetricsCallback()  

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"batch_size": 32,
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
                     "p_dropout": trial.suggest_float("p_drop", 0, 1),
                     "lr_decay": trial.suggest_float("lr_decay", 0.7, 1),
                     "n_hidden": trial.suggest_int("n_hidden", 10, 1000),
                     #"lstm_hidden_size": trial.suggest_int("lstm_hidden_size", 10, 1000),
                     "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1),
                     "normal_spectrogram": True
                    }    

    model = AblationModelPLModule(trial_hparams)
    
    # create a trainer
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(config.LOG_DIR, name=type(model.model).__name__),
        max_epochs=20,                                                               
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, cb],                                                   # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="validation_acc"), # early stopping
    )
    
    datasetHandler.load(model, 'training')#, dataset_id=config.DATASET_CONTROL)
    datasetHandler.load(model, 'validation')#,  dataset_id=config.DATASET_CONTROL)
    trainer.fit(model)

    # save model
    model.save("saved_modelsq/"+'{}.p'.format(trial.number))

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["validation_acc"]

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner, study_name='ablationNormal_study', storage='sqlite:///ablationNormal_study.db', load_if_exists=True)

[I 2020-07-20 18:22:29,488] A new study created with name: ablationNormal_study


In [6]:
#!mkdir saved_modelsq

### Run Search

In [7]:
study.optimize(objective, n_trials=2000, timeout=1.5*21600) #9h

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Loading cached training data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_8k/
Loading cached validation data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_8k/


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name           | Type          | Params
---------------------------------------------
0  | model          | AblationModel | 124 K 
1  | model.convs    | Sequential    | 114 K 
2  | model.convs.0  | BatchNorm2d   | 2     
3  | model.convs.1  | Conv2d        | 1 K   
4  | model.convs.2  | BatchNorm2d   | 20    
5  | model.convs.3  | PReLU         | 1     
6  | model.convs.4  | MaxPool2d     | 0     
7  | model.convs.5  | Dropout       | 0     
8  | model.convs.6  | Conv2d        | 20 K  
9  | model.convs.7  | BatchNorm2d   | 40    
10 | model.convs.8  | PReLU         | 1     
11 | model.convs.9  | MaxPool2d     | 0

Val-Acc=0.016004742145820983


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

Detected KeyboardInterrupt, attempting graceful shutdown...



Saved model to "saved_modelsq/82.p"


Exception ignored in: <function tqdm.__del__ at 0x7f53edc16550>
Traceback (most recent call last):
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/tqdm/std.py", line 1085, in __del__
    def __del__(self):
KeyboardInterrupt: 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-ce1c854bb182>", line 1, in <module>
    study.optimize(objective, n_trials=2000, timeout=1.5*21600) #9h
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/optuna/study.py", line 338, in optimize
    self._optimize_sequential(
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/optuna/study.py", line 682, in _optimize_sequential
    self._run_trial_and_callbacks(func, catch, callbacks, gc_after_trial)
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/optuna/study.py", line 713, in _run_trial_and_callbacks
    trial = self._run_trial(func, catch, gc_after_t

KeyboardInterrupt: 

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
best_trial = study.best_trial
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))